<a href="https://colab.research.google.com/github/khanhbaochau12/khanhbaochau12/blob/main/TRAIN_SpkYLv8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/BICLab/SpikeYOLO.git


fatal: destination path 'SpikeYOLO' already exists and is not an empty directory.


In [ ]:
%cd SpikeYOLO


/content/SpikeYOLO


In [ ]:
!pip install -r requirements.txt


In [ ]:
import torch
print(torch.cuda.device_count())  # Số lượng GPU
print(torch.cuda.is_available())  # GPU có hoạt động không

1
True


In [ ]:
!pip install torchinfo

In [ ]:
!pip install spikingjelly

In [ ]:
!pip install timm

In [ ]:
!pip install einops

In [ ]:
from google.colab import files
files.view('/content/SpikeYOLO/train.py')


<IPython.core.display.Javascript object>

In [ ]:
# Tạo thư mục chứa dữ liệu COCO (chỉ một phần)
!mkdir -p /content/coco_subset/images /content/coco_subset/labels
%cd /content/coco_subset


/content/coco_subset


In [ ]:
!ls -R /content/


/content/:
coco_subset  sample_data  SpikeYOLO

/content/coco_subset:
annotations  coco.yaml	images	labels

/content/coco_subset/annotations:
annotations

/content/coco_subset/annotations/annotations:
captions_train2017.json  instances_train2017.json  person_keypoints_train2017.json
captions_val2017.json	 instances_val2017.json    person_keypoints_val2017.json

/content/coco_subset/images:
train2017

/content/coco_subset/images/train2017:
000000000009.jpg  000000000109.jpg  000000000241.jpg  000000000357.jpg	000000000443.jpg
000000000025.jpg  000000000110.jpg  000000000247.jpg  000000000359.jpg	000000000446.jpg
000000000030.jpg  000000000113.jpg  000000000250.jpg  000000000360.jpg	000000000450.jpg
000000000034.jpg  000000000127.jpg  000000000257.jpg  000000000368.jpg	000000000459.jpg
000000000036.jpg  000000000133.jpg  000000000260.jpg  000000000370.jpg	000000000471.jpg
000000000042.jpg  000000000136.jpg  000000000263.jpg  000000000382.jpg	000000000472.jpg
000000000049.jpg  00000000013

In [ ]:
!cat /content/coco_subset/coco.yaml


path: /content/coco_subset
train: images/train2017
val: images/train2017
nc: 80
names: ["person", "bicycle", "car", "motorcycle", ...]


In [ ]:
%%writefile /content/coco_subset/coco.yaml
path: /content/coco_subset
train: images
val: images
nc: 80
names: ["person", "bicycle", "car", "motorcycle", "airplane", "bus", "train", "truck", "boat", "traffic light",
        "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat", "dog", "horse", "sheep", "cow",
        "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella", "handbag", "tie", "suitcase", "frisbee",
        "skis", "snowboard", "sports ball", "kite", "baseball bat", "baseball glove", "skateboard", "surfboard",
        "tennis racket", "bottle", "wine glass", "cup", "fork", "knife", "spoon", "bowl", "banana", "apple",
        "sandwich", "orange", "broccoli", "carrot", "hot dog", "pizza", "donut", "cake", "chair", "couch",
        "potted plant", "bed", "dining table", "toilet", "tv", "laptop", "mouse", "remote", "keyboard",
        "cell phone", "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase",
        "scissors", "teddy bear", "hair drier", "toothbrush"]


Overwriting /content/coco_subset/coco.yaml


In [ ]:
%%writefile /content/coco_subset/coco.yaml
path: /content/coco_subset
train: images/train2017
val: images/val2017
nc: 80
names: ["person", "bicycle", "car", "motorcycle", ...]  # Danh sách 80 lớp COCO


Overwriting /content/coco_subset/coco.yaml


In [ ]:
!apt-get install unzip -y


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unzip is already the newest version (6.0-26ubuntu3.2).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [ ]:
# Tải annotations (để trích xuất danh sách ảnh)
!wget -q http://images.cocodataset.org/annotations/annotations_trainval2017.zip
!unzip -q annotations_trainval2017.zip -d annotations
!rm annotations_trainval2017.zip

# Tải chỉ 100 ảnh từ tập train2017 (~500MB thay vì 18GB)
!wget -q --show-progress http://images.cocodataset.org/zips/train2017.zip
!unzip -q train2017.zip -d images/
!rm train2017.zip
!ls images/train2017 | head -100 | xargs -I {} mv images/train2017/{} images/
!rm -rf images/train2017  # Xóa phần ảnh không dùng để tiết kiệm dung lượng


replace annotations/annotations/instances_train2017.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace annotations/annotations/instances_val2017.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace annotations/annotations/captions_train2017.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
train2017.zip       100%[===================>]  18.01G  59.0MB/s    in 5m 44s  
replace images/train2017/000000000089.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace images/train2017/000000000450.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace images/train2017/000000000459.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
y


In [ ]:
!ls /content/coco_subset


annotations  coco.yaml	images	labels


In [ ]:
!ls /content/


coco_subset  sample_data  SpikeYOLO


In [ ]:
from google.colab import files
files.view('/content/coco_subset/coco.yaml')

<IPython.core.display.Javascript object>

In [ ]:
!cat /content/coco_subset/coco.yaml



path: /content/coco_subset
train: images/train2017
val: images/val2017
nc: 80
names: ["person", "bicycle", "car", "motorcycle", ...]  # Danh sách 80 lớp COCO


In [ ]:
!ls -R /content/coco_subset/images


/content/coco_subset/images:
000000000009.jpg  000000000109.jpg  000000000241.jpg  000000000357.jpg	000000000443.jpg
000000000025.jpg  000000000110.jpg  000000000247.jpg  000000000359.jpg	000000000446.jpg
000000000030.jpg  000000000113.jpg  000000000250.jpg  000000000360.jpg	000000000450.jpg
000000000034.jpg  000000000127.jpg  000000000257.jpg  000000000368.jpg	000000000459.jpg
000000000036.jpg  000000000133.jpg  000000000260.jpg  000000000370.jpg	000000000471.jpg
000000000042.jpg  000000000136.jpg  000000000263.jpg  000000000382.jpg	000000000472.jpg
000000000049.jpg  000000000138.jpg  000000000283.jpg  000000000384.jpg	000000000474.jpg
000000000061.jpg  000000000142.jpg  000000000294.jpg  000000000387.jpg	000000000486.jpg
000000000064.jpg  000000000143.jpg  000000000307.jpg  000000000389.jpg	000000000488.jpg
000000000071.jpg  000000000144.jpg  000000000308.jpg  000000000394.jpg	000000000490.jpg
000000000072.jpg  000000000149.jpg  000000000309.jpg  000000000395.jpg	000000000491.jpg
000

In [ ]:
!mkdir -p /content/coco_subset/images/train2017
!mv /content/coco_subset/images/*.jpg /content/coco_subset/images/train2017/


In [ ]:
%%writefile /content/coco_subset/coco.yaml
path: /content/coco_subset
train: images/train2017
val: images/train2017  # Nếu chưa có val, có thể dùng chung train
nc: 80
names: ["person", "bicycle", "car", "motorcycle", ...]


Overwriting /content/coco_subset/coco.yaml


In [ ]:
!cat /content/coco_subset/coco.yaml


path: /content/coco_subset
train: images/train2017
val: images/train2017  # Nếu chưa có val, có thể dùng chung train
nc: 80
names: ["person", "bicycle", "car", "motorcycle", ...]


In [ ]:
%%writefile /content/coco_subset/coco.yaml
path: /content/coco_subset
train: images/train2017
val: images/train2017
nc: 80
names: ["person", "bicycle", "car", "motorcycle", ...]


Overwriting /content/coco_subset/coco.yaml


In [ ]:
!rm -rf /root/.config/Ultralytics


In [ ]:
!python train.py --data /content/coco_subset/coco.yaml --epochs 100 --device 0


python3: can't open file '/content/coco_subset/train.py': [Errno 2] No such file or directory


In [ ]:
!find /content -name "train.py"


/content/SpikeYOLO/ultralytics/models/yolo/classify/train.py
/content/SpikeYOLO/ultralytics/models/yolo/segment/train.py
/content/SpikeYOLO/ultralytics/models/yolo/detect/train.py
/content/SpikeYOLO/ultralytics/models/yolo/pose/train.py
/content/SpikeYOLO/ultralytics/models/rtdetr/train.py
/content/SpikeYOLO/train.py
/content/SpikeYOLO/SpikeYOLO_for_Gen1/DESC/RENet/src/train.py
/content/SpikeYOLO/SpikeYOLO_for_Gen1/ultralytics/models/yolo/classify/train.py
/content/SpikeYOLO/SpikeYOLO_for_Gen1/ultralytics/models/yolo/segment/train.py
/content/SpikeYOLO/SpikeYOLO_for_Gen1/ultralytics/models/yolo/detect/train.py
/content/SpikeYOLO/SpikeYOLO_for_Gen1/ultralytics/models/yolo/pose/train.py
/content/SpikeYOLO/SpikeYOLO_for_Gen1/ultralytics/models/rtdetr/train.py
/content/SpikeYOLO/SpikeYOLO_for_Gen1/train.py


In [ ]:
!python /content/SpikeYOLO/train.py --data /content/coco_subset/coco.yaml --epochs 100 --device 0


2025-02-18 13:20:41.834875: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-18 13:20:41.855113: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-18 13:20:41.861320: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/usr/local/lib/python3.10/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/usr/local/lib/python3.10/dist-packages/timm/models/registry.py:4: FutureWarning: Importin

In [ ]:
yaml_path = "/content/coco_subset/coco.yaml"

coco_classes = [
    "person", "bicycle", "car", "motorcycle", "airplane", "bus", "train", "truck",
    "boat", "traffic light", "fire hydrant", "stop sign", "parking meter", "bench",
    "bird", "cat", "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra",
    "giraffe", "backpack", "umbrella", "handbag", "tie", "suitcase", "frisbee",
    "skis", "snowboard", "sports ball", "kite", "baseball bat", "baseball glove",
    "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
    "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange",
    "broccoli", "carrot", "hot dog", "pizza", "donut", "cake", "chair", "couch",
    "potted plant", "bed", "dining table", "toilet", "TV", "laptop", "mouse",
    "remote", "keyboard", "cell phone", "microwave", "oven", "toaster", "sink",
    "refrigerator", "book", "clock", "vase", "scissors", "teddy bear", "hair drier",
    "toothbrush"
]

yaml_content = f"nc: 80\nnames: {coco_classes}\n"

with open(yaml_path, "w") as f:
    f.write(yaml_content)

print("✅ File coco.yaml đã được cập nhật thành công!")


✅ File coco.yaml đã được cập nhật thành công!


In [ ]:
!python /content/SpikeYOLO/train.py --data /content/coco_subset/coco.yaml --epochs 100 --device 0


2025-02-18 13:23:48.806119: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-18 13:23:48.826739: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-18 13:23:48.833029: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/usr/local/lib/python3.10/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/usr/local/lib/python3.10/dist-packages/timm/models/registry.py:4: FutureWarning: Importin

In [ ]:
!ls -lh /content/coco_subset/coco.yaml


-rw-r--r-- 1 root root 876 Feb 18 13:23 /content/coco_subset/coco.yaml


In [ ]:
!cat /content/coco_subset/coco.yaml


nc: 80
names: ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'TV', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']


In [ ]:
yaml_path = "/content/coco_subset/coco.yaml"

yaml_content = """path: /content/coco_subset
train: images/train2017
val: images/val2017
nc: 80
names: ["person", "bicycle", "car", "motorcycle", "airplane", "bus", "train", "truck",
        "boat", "traffic light", "fire hydrant", "stop sign", "parking meter", "bench",
        "bird", "cat", "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra",
        "giraffe", "backpack", "umbrella", "handbag", "tie", "suitcase", "frisbee",
        "skis", "snowboard", "sports ball", "kite", "baseball bat", "baseball glove",
        "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
        "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange",
        "broccoli", "carrot", "hot dog", "pizza", "donut", "cake", "chair", "couch",
        "potted plant", "bed", "dining table", "toilet", "TV", "laptop", "mouse",
        "remote", "keyboard", "cell phone", "microwave", "oven", "toaster", "sink",
        "refrigerator", "book", "clock", "vase", "scissors", "teddy bear", "hair drier",
        "toothbrush"]
"""

with open(yaml_path, "w") as f:
    f.write(yaml_content)

print("✅ File coco.yaml đã được cập nhật thành công!")


✅ File coco.yaml đã được cập nhật thành công!


In [ ]:
!cat /content/coco_subset/coco.yaml


path: /content/coco_subset
train: images/train2017
val: images/val2017
nc: 80
names: ["person", "bicycle", "car", "motorcycle", "airplane", "bus", "train", "truck",
        "boat", "traffic light", "fire hydrant", "stop sign", "parking meter", "bench",
        "bird", "cat", "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra",
        "giraffe", "backpack", "umbrella", "handbag", "tie", "suitcase", "frisbee",
        "skis", "snowboard", "sports ball", "kite", "baseball bat", "baseball glove",
        "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
        "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange",
        "broccoli", "carrot", "hot dog", "pizza", "donut", "cake", "chair", "couch",
        "potted plant", "bed", "dining table", "toilet", "TV", "laptop", "mouse",
        "remote", "keyboard", "cell phone", "microwave", "oven", "toaster", "sink",
        "refrigerator", "book", "clock", "vase", "scissors", "teddy bear", "

In [ ]:
!python /content/SpikeYOLO/train.py --data /content/coco_subset/coco.yaml --epochs 100 --device 0


2025-02-18 13:32:59.366989: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-18 13:32:59.387453: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-18 13:32:59.393618: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/usr/local/lib/python3.10/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/usr/local/lib/python3.10/dist-packages/timm/models/registry.py:4: FutureWarning: Importin

In [ ]:
!ls -lh /content/coco_subset/images/


total 4.0K
drwxr-xr-x 2 root root 4.0K Feb 18 13:18 train2017


In [ ]:
!mkdir -p /content/coco_subset/images/val2017


In [ ]:
!ls -lh /content/coco_subset/images/


total 8.0K
drwxr-xr-x 2 root root 4.0K Feb 18 13:18 train2017
drwxr-xr-x 2 root root 4.0K Feb 18 13:39 val2017


In [ ]:
!cat /content/coco_subset/coco.yaml


path: /content/coco_subset
train: images/train2017
val: images/val2017
nc: 80
names: ["person", "bicycle", "car", "motorcycle", "airplane", "bus", "train", "truck",
        "boat", "traffic light", "fire hydrant", "stop sign", "parking meter", "bench",
        "bird", "cat", "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra",
        "giraffe", "backpack", "umbrella", "handbag", "tie", "suitcase", "frisbee",
        "skis", "snowboard", "sports ball", "kite", "baseball bat", "baseball glove",
        "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
        "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange",
        "broccoli", "carrot", "hot dog", "pizza", "donut", "cake", "chair", "couch",
        "potted plant", "bed", "dining table", "toilet", "TV", "laptop", "mouse",
        "remote", "keyboard", "cell phone", "microwave", "oven", "toaster", "sink",
        "refrigerator", "book", "clock", "vase", "scissors", "teddy bear", "

In [ ]:
!python /content/SpikeYOLO/train.py --data /content/coco_subset/coco.yaml --epochs 100 --device 0


2025-02-18 13:40:55.565301: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-18 13:40:55.585376: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-18 13:40:55.591490: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/usr/local/lib/python3.10/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/usr/local/lib/python3.10/dist-packages/timm/models/registry.py:4: FutureWarning: Importin